## RAG Process 

- 사전작업(Indexing): 문서 로드 및 VectorDB에 저장
1. 문서로드
2. 분할(Split)
3. 임베딩(Embedding)
4. 벡터DB(VectorStore)

- RAG수행: 뭍서 검색 및 결과 도출
5. 검색(Retrieval)
6. 프롬프트(Prompt)
7. 모델(LLM)
8. 결과(Output)

셀 나눔: ctrl + shift + '-'
셀 합치기 alt + window + j

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_teddynote import logging

logging.langsmith("CH12-RAG")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH12-RAG


In [3]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate

from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings

In [4]:
# from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [5]:
docs = PyPDFDirectoryLoader("data")
docs = docs.load()

In [6]:
print(len(docs))
print(docs[1].page_content)

23
2023 년 12월호
Ⅰ. 인공지능 산업 동향 브리프
 1. 정책/법제 
   ▹ 미국, 안전하고 신뢰할 수 있는 AI 개발과 사용에 관한 행정명령 발표  ························· 1
   ▹ G7, 히로시마 AI 프로세스를 통해 AI 기업 대상 국제 행동강령에 합의··························· 2
   ▹ 영국 AI 안전성 정상회의에 참가한 28개국, AI 위험에 공동 대응 선언··························· 3
   ▹ 미국 법원, 예술가들이 생성 AI 기업에 제기한 저작권 소송 기각····································· 4
   ▹ 미국 연방거래위원회 , 저작권청에 소비자 보호와 경쟁 측면의 AI 의견서 제출················· 5
   ▹ EU AI 법 3자 협상, 기반모델 규제 관련 견해차로 난항··················································· 6
 
 2. 기업/산업 
   ▹ 미국 프런티어 모델 포럼, 1,000 만 달러 규모의 AI 안전 기금 조성································ 7
   ▹ 코히어 , 데이터 투명성 확보를 위한 데이터 출처 탐색기 공개  ······································· 8
   ▹ 알리바바 클라우드 , 최신 LLM ‘통이치엔원 2.0’ 공개 ······················································ 9
   ▹ 삼성전자 , 자체 개발 생성 AI ‘삼성 가우스 ’ 공개 ··························································· 10
   ▹ 구글, 앤스로픽에 20억 달러 투자로 생성 AI 협력 강화 ················································ 11
   ▹ IDC, 2027 년 AI 소프트웨어 

In [7]:
docs[1]

Document(page_content='2023 년 12월호\nⅠ. 인공지능 산업 동향 브리프\n 1. 정책/법제 \n   ▹ 미국, 안전하고 신뢰할 수 있는 AI 개발과 사용에 관한 행정명령 발표  ························· 1\n   ▹ G7, 히로시마 AI 프로세스를 통해 AI 기업 대상 국제 행동강령에 합의··························· 2\n   ▹ 영국 AI 안전성 정상회의에 참가한 28개국, AI 위험에 공동 대응 선언··························· 3\n   ▹ 미국 법원, 예술가들이 생성 AI 기업에 제기한 저작권 소송 기각····································· 4\n   ▹ 미국 연방거래위원회 , 저작권청에 소비자 보호와 경쟁 측면의 AI 의견서 제출················· 5\n   ▹ EU AI 법 3자 협상, 기반모델 규제 관련 견해차로 난항··················································· 6\n \n 2. 기업/산업 \n   ▹ 미국 프런티어 모델 포럼, 1,000 만 달러 규모의 AI 안전 기금 조성································ 7\n   ▹ 코히어 , 데이터 투명성 확보를 위한 데이터 출처 탐색기 공개  ······································· 8\n   ▹ 알리바바 클라우드 , 최신 LLM ‘통이치엔원 2.0’ 공개 ······················································ 9\n   ▹ 삼성전자 , 자체 개발 생성 AI ‘삼성 가우스 ’ 공개 ··························································· 10\n   ▹ 구글, 앤스로픽에 20억 달러 투자로 생성 AI 협력 강화 ···········································

In [8]:
docs[1].metadata

{'source': 'data\\SPRI_AI_Brief_2023년12월호_F.pdf', 'page': 1}

In [9]:
from langchain_community.document_loaders import PyMuPDFLoader

docs = PyMuPDFLoader("data/SPRI_AI_Brief_2023년12월호_F.pdf")
docs = docs.load()

In [10]:
# metadata는 어떤 loader를 쓰느냐에 따라서 다르다.
docs[1].metadata

{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pdf',
 'file_path': 'data/SPRI_AI_Brief_2023년12월호_F.pdf',
 'page': 1,
 'total_pages': 23,
 'format': 'PDF 1.4',
 'title': '',
 'author': 'dj',
 'subject': '',
 'keywords': '',
 'creator': 'Hwp 2018 10.0.0.13462',
 'producer': 'Hancom PDF 1.3.0.542',
 'creationDate': "D:20231208132838+09'00'",
 'modDate': "D:20231208132838+09'00'",
 'trapped': ''}

In [11]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)

In [12]:
documents = text_splitter.split_documents(docs)

In [13]:
len(documents)

43

In [14]:
print(documents[1].page_content)

2023년 12월호
Ⅰ. 인공지능 산업 동향 브리프
 1. 정책/법제 
   ▹ 미국, 안전하고 신뢰할 수 있는 AI 개발과 사용에 관한 행정명령 발표  ························· 1
   ▹ G7, 히로시마 AI 프로세스를 통해 AI 기업 대상 국제 행동강령에 합의··························· 2
   ▹ 영국 AI 안전성 정상회의에 참가한 28개국, AI 위험에 공동 대응 선언··························· 3
   ▹ 미국 법원, 예술가들이 생성 AI 기업에 제기한 저작권 소송 기각····································· 4
   ▹ 미국 연방거래위원회, 저작권청에 소비자 보호와 경쟁 측면의 AI 의견서 제출················· 5
   ▹ EU AI 법 3자 협상, 기반모델 규제 관련 견해차로 난항··················································· 6
 
 2. 기업/산업 
   ▹ 미국 프런티어 모델 포럼, 1,000만 달러 규모의 AI 안전 기금 조성································ 7
   ▹ 코히어, 데이터 투명성 확보를 위한 데이터 출처 탐색기 공개  ······································· 8
   ▹ 알리바바 클라우드, 최신 LLM ‘통이치엔원 2.0’ 공개 ······················································ 9
   ▹ 삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개 ··························································· 10
   ▹ 구글, 앤스로픽에 20억 달러 투자로 생성 AI 협력 강화 ················································ 11


In [15]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [16]:
# 성능이 떨어진다.. retriever를 제대로 하지 못한다.
# embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [17]:
vector_store = FAISS.from_documents(documents, embeddings)

In [18]:
retriever = vector_store.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002971594FA90>)

In [19]:
retriever.invoke("삼성전자가 계발한 생성형 AI이름은 무엇인가요?")

[Document(page_content='SPRi AI Brief |  \n2023-12월호\n10\n삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개\nn 삼성전자가 온디바이스에서 작동 가능하며 언어, 코드, 이미지의 3개 모델로 구성된 자체 개발 생성 \nAI 모델 ‘삼성 가우스’를 공개\nn 삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획으로, 온디바이스 작동이 가능한 \n삼성 가우스는 외부로 사용자 정보가 유출될 위험이 없다는 장점을 보유\nKEY Contents\n£ 언어, 코드, 이미지의 3개 모델로 구성된 삼성 가우스, 온디바이스 작동 지원\nn 삼성전자가 2023년 11월 8일 열린 ‘삼성 AI 포럼 2023’ 행사에서 자체 개발한 생성 AI 모델 \n‘삼성 가우스’를 최초 공개\n∙정규분포 이론을 정립한 천재 수학자 가우스(Gauss)의 이름을 본뜬 삼성 가우스는 다양한 상황에 \n최적화된 크기의 모델 선택이 가능\n∙삼성 가우스는 라이선스나 개인정보를 침해하지 않는 안전한 데이터를 통해 학습되었으며, \n온디바이스에서 작동하도록 설계되어 외부로 사용자의 정보가 유출되지 않는 장점을 보유\n∙삼성전자는 삼성 가우스를 활용한 온디바이스 AI 기술도 소개했으며, 생성 AI 모델을 다양한 제품에 \n단계적으로 탑재할 계획\nn 삼성 가우스는 △텍스트를 생성하는 언어모델 △코드를 생성하는 코드 모델 △이미지를 생성하는 \n이미지 모델의 3개 모델로 구성\n∙언어 모델은 클라우드와 온디바이스 대상 다양한 모델로 구성되며, 메일 작성, 문서 요약, 번역 업무의 \n처리를 지원\n∙코드 모델 기반의 AI 코딩 어시스턴트 ‘코드아이(code.i)’는 대화형 인터페이스로 서비스를 제공하며 \n사내 소프트웨어 개발에 최적화\n∙이미지 모델은 창의적인 이미지를 생성하고 기존 이미지를 원하는 대로 바꿀 수 있도록 지원하며 \n저해상도 이미지의 고해상도 전환도 지원\nn IT 전문지 테크리퍼블릭(TechRepublic)은 온디바이스 AI가 주요 기

In [20]:
retriever.invoke("구글은 앤스로픽에 얼마를 투자했나요?")

[Document(page_content='1. 정책/법제  \n2. 기업/산업 \n3. 기술/연구 \n 4. 인력/교육\n구글, 앤스로픽에 20억 달러 투자로 생성 AI 협력 강화 \nn 구글이 앤스로픽에 최대 20억 달러 투자에 합의하고 5억 달러를 우선 투자했으며, 앤스로픽은 \n구글과 클라우드 서비스 사용 계약도 체결\nn 3대 클라우드 사업자인 구글, 마이크로소프트, 아마존은 차세대 AI 모델의 대표 기업인 \n앤스로픽 및 오픈AI와 협력을 확대하는 추세\nKEY Contents\n£ 구글, 앤스로픽에 최대 20억 달러 투자 합의 및 클라우드 서비스 제공\nn 구글이 2023년 10월 27일 앤스로픽에 최대 20억 달러를 투자하기로 합의했으며, 이 중 5억 \n달러를 우선 투자하고 향후 15억 달러를 추가로 투자할 방침\n∙구글은 2023년 2월 앤스로픽에 이미 5억 5,000만 달러를 투자한 바 있으며, 아마존도 지난 9월 \n앤스로픽에 최대 40억 달러의 투자 계획을 공개\n∙한편, 2023년 11월 8일 블룸버그 보도에 따르면 앤스로픽은 구글의 클라우드 서비스 사용을 위해 \n4년간 30억 달러 규모의 계약을 체결\n∙오픈AI 창업자 그룹의 일원이었던 다리오(Dario Amodei)와 다니엘라 아모데이(Daniela Amodei) \n남매가 2021년 설립한 앤스로픽은 챗GPT의 대항마 ‘클로드(Claude)’ LLM을 개발\nn 아마존과 구글의 앤스로픽 투자에 앞서, 마이크로소프트는 차세대 AI 모델의 대표 주자인 오픈\nAI와 협력을 확대\n∙마이크로소프트는 오픈AI에 앞서 투자한 30억 달러에 더해 2023년 1월 추가로 100억 달러를 \n투자하기로 하면서 오픈AI의 지분 49%를 확보했으며, 오픈AI는 마이크로소프트의 애저(Azure) \n클라우드 플랫폼을 사용해 AI 모델을 훈련\n£ 구글, 클라우드 경쟁력 강화를 위해 생성 AI 투자 확대\nn 구글은 수익률이 높은 클라우드 컴퓨팅 시장에서 아마존과 마이크로소프트를 따라잡고자 생성 A

In [21]:
prompt = PromptTemplate.from_template(
    """
You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Answer in Korean

#Question:
{question}
#Context:
{context}

#Anser:
"""
)

In [22]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro", temperature=0)

In [23]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [24]:
chain.invoke("삼성전자가 자체 개발한 ai의 이름은?")

'삼성전자가 자체 개발한 AI의 이름은 **삼성 가우스**입니다. \n'

In [25]:
from langchain_teddynote.messages import stream_response

question = "구글은 앤스로픽에 얼마를 투자했나요?"
response = chain.stream(question)
stream_response(response)

구글은 앤스로픽에 최대 20억 달러를 투자하기로 합의했습니다. 5억 달러는 이미 투자되었고, 향후 15억 달러를 추가로 투자할 예정입니다. 


In [26]:
chain.invoke("구글은 앤스로픽에 얼마를 투자했나요??")

'구글은 앤스로픽에 최대 20억 달러를 투자하기로 합의했습니다. 이 중 5억 달러를 우선 투자했고, 향후 15억 달러를 추가로 투자할 예정입니다. \n'

In [27]:
chain.invoke("구글과 앤스로픽의 관계는??")

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..


'구글은 생성 AI 스타트업인 앤스로픽에 최대 20억 달러를 투자하고 클라우드 서비스를 제공하는 협력 관계를 맺고 있습니다. 앤스로픽은 구글 클라우드 서비스를 사용하고, 구글은 앤스로픽의 성장과 기술 발전을 지원합니다. \n'

In [28]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [29]:
vector_store2 = FAISS.from_documents(documents, embeddings)
retriever2 = vector_store2.as_retriever()

In [30]:
retriever2.invoke("삼성전자가 자체 개발한 ai의 이름은?")

[Document(page_content='n 연구진은 상기 원칙에 따라 AI를 성능에 따라 0~5단계와 광범위한 목적에 활용될 수 있는 범용 AI 및 특정 \n과업에 활용되는 특수 AI로 분류했으며, 특수 AI에서는 5단계까지 달성되었으나, 범용 AI는 현재 1단계 수준\n성능\n특수 AI 예시\n범용 AI 예시\n0단계: AI 아님\n계산기 소프트웨어, 컴파일러\n아마존 메커니컬 터크\n1단계: 신진(숙련되지 않은 인간)\nGOFAI(Good Old Fashioned Artificial Intelligence) \n챗GPT, 바드, 라마2\n2단계: 유능(숙련된 인간의 50% 이상)\n스마트 스피커(애플 시리, 아마존 알렉사, 구글 \n어시스턴트), IBM 왓슨 \n미달성\n3단계: 전문가(숙련된 인간의 90% 이상)\n문법 교정기(그래머리), 생성 이미지 모델(달리2)\n미달성\n4단계: 거장(숙련된 인간의 99% 이상) \n딥블루, 알파고\n미달성\n5단계: 초인간(인간을 100% 능가)\n알파폴드, 알파제로, 스톡피시\n미달성\n<구글 딥마인드의 범용 AI 분류 프레임워크> \n☞ 출처 : Arxiv.org, Levels of AGI: Operationalizing Progress on the Path to AGI, 2023.11.04.', metadata={'source': 'data/SPRI_AI_Brief_2023년12월호_F.pdf', 'file_path': 'data/SPRI_AI_Brief_2023년12월호_F.pdf', 'page': 18, 'total_pages': 23, 'format': 'PDF 1.4', 'title': '', 'author': 'dj', 'subject': '', 'keywords': '', 'creator': 'Hwp 2018 10.0.0.13462', 'producer': 'Hancom PDF 1.3.0.542', 'creationDate': "D:20231208132838+09'00'", 'modDate': 

In [31]:
retriever2.invoke("구글은 앤스로픽에 얼마를 투자했나요?")

[Document(page_content='지침을 제공하며, 보건복지 부문에서 책임 있는 AI 배포와 사용을 위한 전략을 마련 \nn (소비자 보호와 근로자 지원) 의료 분야에서 책임 있는 AI 사용을 촉진하고 맞춤형 개인교습 등 학교 \n내 AI 교육 도구 관련 자원을 개발하며, AI로 인한 근로자 피해를 완화하고 이점을 극대화하는 원칙과 \n모범사례를 마련\nn (혁신과 경쟁 촉진) 국가AI연구자원(National Artificial Intelligence Research Resource, NAIRR)*을 \n통해 미국 전역의 AI 연구를 촉진하고, 중소기업과 개발자에 기술과 인프라를 지원\n* 국가 차원에서 AI 연구 인프라를 확충해 더 많은 AI 연구자에게 인프라를 지원하는 프로그램 \n∙비자 기준과 인터뷰 절차의 현대화와 간소화로 AI 관련 주요 분야의 전문 지식을 갖춘 외국인들이 미국에서 \n공부하고 취업할 수 있도록 지원\n☞ 출처 : The White House, Executive Order on the Safe, Secure, and Trustworthy Development and Use of \nArtificial Intelligence (E.O. 14110), 2023.10.30.', metadata={'source': 'data/SPRI_AI_Brief_2023년12월호_F.pdf', 'file_path': 'data/SPRI_AI_Brief_2023년12월호_F.pdf', 'page': 3, 'total_pages': 23, 'format': 'PDF 1.4', 'title': '', 'author': 'dj', 'subject': '', 'keywords': '', 'creator': 'Hwp 2018 10.0.0.13462', 'producer': 'Hancom PDF 1.3.0.542', 'creationDate': "D:20231208132838+09'00'", 'modDate': "D:20231208132838+09'00'",

In [32]:
chain2 = (
    {"context": retriever2, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [33]:
from langchain_teddynote.messages import stream_response

question = "구글은 앤스로픽에 얼마를 투자했나요?"
response = chain2.stream(question)
stream_response(response)

제공된 문서에는 구글이 앤스로픽에 얼마를 투자했는지에 대한 정보가 없습니다. 죄소하지만 귀하의 질문에 답변할 수 없습니다.

In [1]:
from langchain_teddynote.messages import stream_response

In [2]:
stream_response

<function langchain_teddynote.messages.stream_response(response, return_output=False)>